# 4 - Convolutional Sentiment Analysis

This is a port of pytorch implementation [Convolutional Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb) to TensorFlow 2

In [3]:
!pip install tf-nightly -q
!pip install comet_ml -q

     |████████████████████████████████| 456.5MB 36kB/s 
     |████████████████████████████████| 3.9MB 50.6MB/s 
     |████████████████████████████████| 460kB 63.0MB/s 
     |████████████████████████████████| 2.9MB 64.8MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
ERROR: tb-nightly 2.2.0a20200106 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
     |████████████████████████████████| 174kB 3.4MB/s 
     |████████████████████████████████| 204kB 10.9MB/s 
     |████████████████████████████████| 409kB 9.6MB/s 


In [0]:
import os
import string
import re
from comet_ml import Experiment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Embedding, GlobalAveragePooling1D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
tf.__version__

'2.2.0-dev20200123'

In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [0]:
tf.random.set_seed(31)
np.random.seed(31)

In [0]:
COMET_API_KEY = "N9ue7hxUF8fHmAunfA0EEaYbg"

In [9]:
experiment = Experiment(api_key=COMET_API_KEY, project_name="cnn-sentiment", workspace="dzlab")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/dzlab/cnn-sentiment/6752a1554fb746979b9e7dc50ddd0ab8



In [11]:
dataset = tfds.load('imdb_reviews')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
train_raw, test_raw, unsupervised_raw = dataset['train'], dataset['test'], dataset['unsupervised']

In [0]:
format_dataset = lambda item: (item['text'], item['label'])

train_ds, test_ds = train_raw.map(format_dataset), test_raw.map(format_dataset)

In [0]:
train_ds = train_ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [148]:
for item in train_ds.take(1):
  print(item)
for item in test_ds.take(1):
  print(item)

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b"there is no suspense in this serial! When one episode ends the acting is so shoddy, the effects are so poor and the script is so awful that the last thing on your mind is how Batman and Robin will save the day. No, in fact, the last thing on your mind is watching the next episode! This show is so boring that I can't see how it ever got made, let alone released on DVD! Obviously the effects are not up to par with contemporary Batman films, but even the script is awful. An incoherent babbling mess about some evil professor and a ray gun or something like that, I am not quite sure, because it is too awful to follow. Watch the 60s version, or the 90's versions, or even Batman Begins, just anything over this version!",
       b'One of the more sensible comedies to hit the Hindi film screens. A remake of Priyadarshans 80s Malayalam hit Boeing Boeing, which in turn was a remake of the 60s Hollywoon hit of the same name, Garam Masala elev

In [0]:
unsupervised = train_raw.map(lambda item: item['text'])

In [0]:
vectorize = TextVectorization(
  standardize=custom_standardization,
  max_tokens=10000,
  output_mode='int',
  output_sequence_length=500
)
vectorize.adapt(unsupervised.batch(64))

In [0]:
pd.set_option('display.large_repr', 'truncate')
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 150)
pd.set_option('expand_frame_repr', True)

In [94]:
pd.DataFrame(train_raw.take(5))

,label,text
0,"tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b""As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presente..."
1,"tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b""Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel ..."
2,"tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b""I saw this film on True Movies (which automatically made me sceptical) but actually - it was good. Why? Not because of the amazing plo..."
3,"tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b'This was a wonderfully clever and entertaining movie that I shall never tire of watching many, many times. The casting was magnificent..."
4,"tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like ti..."


We need to clean up the text from things like `<br />`, see [example](https://colab.research.google.com/drive/1RvCnR7h0_l4Ekn5vINWToI9TNJdpUZB3#scrollTo=RToU2ObJvNoj).

In [0]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [0]:
class CNN(tf.keras.Model):
  def __init__(self, max_features, embedding_dim, dropout, filters, filter_sizes, output_dim):
    super(CNN, self).__init__(name='cnn_sentiment')
    self.embed = Embedding(max_features + 1, embedding_dim)
    self.dropout = Dropout(dropout)
    self.convs = [
      Conv2D(filters, kernel_size=(fs, embedding_dim), data_format='channels_first') for fs in filter_sizes
    ]
    self.pool = GlobalAveragePooling1D(data_format='channels_first')
    self.dense = Dense(output_dim, activation='relu') # input_shape=len(filter_sizes) * filters, 

  def call(self, inputs):
    # inputs = [batch, seq_len]
    x = self.embed(inputs)
    # x = [batch, seq_len, embed_dim]
    x = tf.expand_dims(x, 1)
    # x = [batch, 1, seq_len, embed_dim]
    xs = [tf.squeeze(conv(x), axis=3) for conv in self.convs]
    # x = [batch, filters, seq_len - filter_sizes + 1]
    xs = [self.pool(x) for x in xs]
    x = tf.concat(xs, axis=1)
    return self.dense(x)


In [0]:
cnn = CNN(
  max_features=10000,
  embedding_dim=128,
  dropout=0.2,
  filters=32,
  filter_sizes = [3,4,5],
  output_dim = 1
)

In [0]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
predictions = cnn(vectorize(text_input))

model = tf.keras.Model(text_input, predictions)

In [158]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text (InputLayer)            [(None, 1)]               0         
_________________________________________________________________
text_vectorization_10 (TextV (None, 500)               0         
_________________________________________________________________
cnn_sentiment (CNN)          (None, 1)                 1329473   
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [160]:
model.fit(train_ds, validation_data=test_ds, epochs=5, verbose=1, callbacks=[EarlyStopping()])

Epoch 1/5
782/782 [==============================] - 36s 46ms/step - loss: 0.6234 - accuracy: 0.8956 - val_loss: 0.4553 - val_accuracy: 0.8840
Epoch 2/5
782/782 [==============================] - 36s 45ms/step - loss: 0.3937 - accuracy: 0.9199 - val_loss: 0.5890 - val_accuracy: 0.8746


In [161]:
model.predict(test_ds)

array([[0.61076546],
       [0.02746261],
       [0.89037955],
       ...,
       [0.36191127],
       [0.8670504 ],
       [1.5629838 ]], dtype=float32)